## This notebook is designed for the lecture on transport phenomena (29.10.2020)

__As you will see, this notebook consists of four parts:__
- Real data for comparison
- A finite-differences implementation to solve the PDE
- A cell to define the parameter alpha
- A comparison between real values and calculated ones

__First, choose an alpha for section 2. Then, run all sections 0 to 4 to get a result. If you want to change the parameter alpha afterwards, rerun section 2-4 after changing alpha__

You can run the cells either by clicking on the cell and then on Run or press ctrl+enter


## 0. Install packages

This might take a moment, but you have to execute this cell only once. 

In [ ]:
!pip install numpy
!pip install matplotlib

## 1. Real Data

Below is the measured data during the experiment. Your goal will be to find the correct alpha value based on this data set

In [ ]:
# Get Measured Data
# In this cell, the data points from the experiment are stored
# Please don't change anything and run this cell once
import numpy as np

## 2. Paramter

### Define the alpha value in the cell below;  your value should be in the magnitude of $10^{-6}$

In [3]:
# Parameter definition

# Define a value for alpha here :

###################################################################################################################

alpha = 10*10**-6  # [m²/s]

###################################################################################################################


## 3. Finite Differences Implementation 

### Differential Heat Equation 

$\Large \frac{\partial{u(x,y,t)}}{\partial{t}} = \alpha\ \{ \frac{\partial^2{u(x,y,t)}}{\partial^2{x}} + \frac{\partial^2{u(x,y,t)}}{\partial^2{y}} \}$

To incoporate for the outter conditions, constraints need to be defined, both in time and space:

- $ u(x=0,y,t) = T_{hot} $ : Assumption here is that the left part of the ruler is isothermally at temperature $T_{hot}$


- $ u(x\neq 0,y,t=0) = T_{cold} $ : At time $t_0$, the rest of the ruler is cold

### Finite Difference Discretization of Temperature equation
For a specific place, given by $x_n$, $y_n$ and time $t_n$, the temperature $u_{x_n,y_n}^{n+1}$ at the next time step $t_{n+1}$ can be approximated by:

$\Large u_{x_n,y_n}^{t_{n+1}} = u_{x_n,y_n}^{t_n} + \alpha \Delta t ((u_{x_n,y_{n+1}}^{t_n} - 2 u_{x_n,y_{n}}^{t_n}+ u_{x_n,y_{n-1}}^{t_n}) + (u_{x_{n+1},y_{n}}^{t_n} - 2 u_{x_n,y_{n}}^t{t_n} + u_{x_{n-1},y_{n}}^{t_n} )) $

$x_{n+1},y_{n+1}$ and $x_{n-1},y_{n-1}$ are hereby the next and the former neighbour, respectively. 


In [4]:
# helper functions for animation
def init():
    # initalizes figure and sets desgin of image
    global fig, ax, im
    fig = plt.figure(1)
    # image with cmap and color bar
    im = plt.imshow(u_tot[:,:,0],cmap=plt.get_cmap('hot'), vmin=T_cool,vmax=T_hot,extent=[0,w,h,0])
    plt.xlabel('[cm]')
    plt.ylabel('[cm]')
    cbar_ax = fig.add_axes([0.9, 0.15, 0.03, 0.7]) 
    cbar_ax.set_xlabel('$T$ / K', labelpad=20)
    fig.colorbar(im, cax=cbar_ax)
    #initalize image with temperature at time 0
    im.set_data(u_tot[:,:,0])
    plt.close()
    return 

def animate(i):
    # changes image of temperture to time: i*dt 
    global fig, u_tot
    fig.suptitle('{:.1f} min'.format(simulation_time/nframes*i))  #set title above image to current time
    im.set_data(u_tot[:,:,i]) #change temperature to i-th temperature
    return im,

In [9]:
# calculate each temperature(x,y,t) with central difference in space and Euler forward in time

def do_timestep(u):
    for i in range(dnframes):
        u[1:-1, 1:-1] = u[1:-1, 1:-1] + alpha * dt * (
              (u[2:, 1:-1] - 2*u[1:-1, 1:-1] + u[:-2, 1:-1])/dx2
              + (u[1:-1, 2:] - 2*u[1:-1, 1:-1] + u[1:-1, :-2])/dy2 )
        u[0,:] = u[1,:]    # set boundaries to same as neighbour
        u[-1,:] = u[-2,:]  # set boundaries to same as neighbour
        u[:,0] = u[:,1]    # set boundaries to same as neighbour
        u[:,-1] = u[:,-2]  # set boundaries to same as neighbour
        u = get_constraints(u.copy())
    return u

def get_constraints(u):
    u[:,0] = T_hot
    return u

            
    

In [12]:
#geometrical parameter
w = 10       # width [mm]
h = 10       # height [mm]
dx = 0.1      # finite step in x direction [mm]
dy = 0.1     #finite step in y direction [mm]
# temperatures
T_cool = 300   # inital temperature[K]
T_hot = 700   # hot temperature [K]
#time
simulation_time = 50 #[min] 

#number of nodes
nx = int(w/dx) #number of nodes in x direction 
ny = int(h/dy) #number of nodes in y direction

dx2, dy2 = dx*dx, dy*dy  #[mm²]
dt = dx2 * dy2 / (2 * alpha * (dx2 + dy2)) #[ms]

# Number of timesteps
nsteps = int(simulation_time*60/dt*1000)

# Number of frames 
nframes = 40 #to keep computation feasable, only 40 frames are used, regardeless simulation time
dnframes = int(nsteps/nframes) #number of timesteps in a single frame


In [13]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# initialization temperature field at time 0
u0 = T_cool * np.ones((nx, ny)) # initialize field of nx*ny, all cool
u0 = get_constraints(u0) #introduce constraints 

# initalization temperature field over time
u_tot = np.ones([nx,ny,nframes+1]) # initalize all to one
u_tot[:,:,0] = u0 #at timestep zero, equal to initalized heat field 

#iterate through time
for i in range(nframes):
    global u
    u = u_tot[:,:,i] # current temperature field 
    u_tot[:,:,i+1]=do_timestep(u.copy())
    

# animate results
plt.ion()
init()
ani = animation.FuncAnimation(fig, animate, frames = nframes) 
HTML(ani.to_jshtml())

As you will see when you carried out the simulation correclty, with time the heat propagates from the left to the right. The propagation is at the beginning stronger because the temperature gradient is stronger. With time, it propagates more slowly

## 4. Results 

In [ ]:
# You will get your result when executing this cell. The result is the mean squared error, so the squarred error 
# between the model's predicted values and the measured values

#please fill out the google form with your alpha value and the obtained result 

# MSE:
#mse = (np.square(T - realT)).mean(axis=0)
#display('    ')
#display('The mean squarred error (MSE) for alpha = ' + str(alpha) + ' resulted in ' + str(np.round(mse,3)))

## Please submit your result to the google form for comparsion with the rest of the class